# Q3: Importing libraries and Dataset

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\david\04-06-2024 Instacart Basket Analysis'

In [3]:
customer_df = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'))

# Q4: Data Wrangling

In [4]:
customer_df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Things to change:
- make all columns use lowercase letters for consistency
- add an underscore between first and last name
- convert Surnam to surname

In [5]:
customer_df_wrangled = customer_df

In [6]:
# renaming columns
customer_df_wrangled.rename(columns={'First Name': 'first_name', 'Surnam': 'surname', 'Gender': 'gender', 'STATE': 'state', 'Age': 'age'}, inplace = True)

In [7]:
customer_df_wrangled.head()

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# checking to make sure new df is the same length as previous
customer_df.shape

(206209, 10)

In [9]:
customer_df_wrangled.shape

(206209, 10)

# Q5: Data Quality and Consistency Check

## Missing Data Check

In [10]:
customer_df_wrangled.isnull().sum()

user_id             0
first_name      11259
surname             0
gender              0
state               0
age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

There are 11,259 missing first names. The data will most likely be aggregated, so missing names are not a concern. No action needed

In [14]:
customer_df_no_missing = customer_df

## Checking for duplicates

In [15]:
customer_df_no_dups = customer_df_no_missing[customer_df_no_missing.duplicated()]

In [16]:
customer_df_no_dups

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income


In [17]:
# No Duplicate Values

## Checking for mixed type data

In [18]:
df_test = customer_df_no_missing

In [19]:
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

first_name


Mixed type data in first name column. Most likely from missing values.

In [ ]:
customer_df_no_missing['first_name'] = customer_df_no_missing['first_name'].astype('str')

# Q6: Combining customer data with instacart data

In [21]:
df_customer_cleaned = customer_df_no_missing

In [22]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge_4-8.pkl'))

## Before merge:

In [23]:
df_customer_cleaned.shape

(206209, 10)

In [24]:
ords_prods_merge.shape

(32404859, 27)

## Pre-merge checks

df_customer_cleaned and ords_prods_merge will be merged on user_id. The two columns need to be of the same type

In [25]:
df_customer_cleaned.dtypes

user_id          int64
first_name      object
surname         object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [26]:
ords_prods_merge.dtypes

Unnamed: 0.1                     int64
Unnamed: 0_x                     int64
order_id                         int64
user_id                          int64
order_ordinal_number             int64
order_day_of_week                int64
order_hour_of_day                int64
days_since_prior_order         float64
first_order                       bool
product_id                       int64
add_to_cart_order                int64
reordered                        int64
_merge                        category
Unnamed: 0_y                     int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
price_range_loc                 object
busiest_day                     object
busiest_period_of_day           object
max_order                        int64
loyalty_flag                    object
mean_product_price             float64
spending_flag                   object
median_days_between_order

## Big merge

In [27]:
pd.set_option('display.max_columns', None)

In [28]:
ords_prods_merge.drop(columns = '_merge', inplace = True)

In [29]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_ordinal_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,1,1,2398795,1,2,3,7,15.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2,2,473747,1,3,3,12,21.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,3,3,2254736,1,4,4,7,29.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,4,4,431534,1,5,4,15,28.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Most orders - Top 5 Hours,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [30]:
ords_prods_customers = ords_prods_merge.merge(df_customer_cleaned, on = 'user_id', indicator = True)

In [31]:
ords_prods_customers.shape

(32404859, 36)

In [32]:
ords_prods_customers.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_ordinal_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,1,1,2398795,1,2,3,7,15.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2,2,473747,1,3,3,12,21.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,3,3,2254736,1,4,4,7,29.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,4,4,431534,1,5,4,15,28.0,False,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy days,Most orders - Top 5 Hours,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [33]:
ords_prods_customers.value_counts('_merge')

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [34]:
# exporting data frame
ords_prods_customers.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_customers.pkl'))